# Your First RAQA Application

In this notebook, we'll walk you through each of the components that are involved in a simple RAQA application. 

We won't be leveraging any fancy tools, just the OpenAI Python SDK, Numpy, and some classic Python.

This was done with Python 3.11.4.

Let's look at a rather complicated looking visual representation of a basic RAQA application.

<img src="https://i.imgur.com/LCNkd1A.png" />

### Imports and Utility 

We're just doing some imports and enabling `async` to work within the Jupyter environment here, nothing too crazy!

In [1]:
from aimakerspace.text_utils import TextFileLoader, CharacterTextSplitter
from aimakerspace.vectordatabase import VectorDatabase
import asyncio

In [2]:
import nest_asyncio
nest_asyncio.apply()

# Documents

We'll be concerning ourselves with this part of the flow in the following section:

<img src="https://i.imgur.com/wBYB2x3.png" />

### Loading Source Documents

So, first things first, we need some documents to work with. 

While we could work directly with the `.txt` files (or whatever file-types you wanted to extend this to) we can instead do some batch processing of those documents at the beginning in order to store them in a more machine compatible format. 

In this case, we're going to parse our text file into a single document in memory.

Let's look at the relevant bits of the `TextFileLoader` class:

```python
def load_file(self):
        with open(self.path, "r", encoding=self.encoding) as f:
            self.documents.append(f.read())
```

We're simply loading the document using the built in `open` method, and storing that output in our `self.documents` list.


In [3]:
text_loader = TextFileLoader("data/KingLear.txt")
documents = text_loader.load_documents()
len(documents)

1

In [9]:
print(documents[0][-500:])

th endured so long:
He but usurp'd his life.
ALBANY
Bear them from hence. Our present business
Is general woe.
To KENT and EDGAR

Friends of my soul, you twain
Rule in this realm, and the gored state sustain.
KENT
I have a journey, sir, shortly to go;
My master calls me, I must not say no.
ALBANY
The weight of this sad time we must obey;
Speak what we feel, not what we ought to say.
The oldest hath borne most: we that are young
Shall never see so much, nor live so long.
Exeunt, with a dead march


In [4]:
print(documents[0][:100])

ACT I
SCENE I. King Lear's palace.
Enter KENT, GLOUCESTER, and EDMUND
KENT
I thought the king had m


### Splitting Text Into Chunks

As we can see, there is one document - and it's the entire text of King Lear.

We'll want to chunk the document into smaller parts so it's easier to pass the most relevant snippets to the LLM. 

There is no fixed way to split/chunk documents - and you'll need to rely on some intuition as well as knowing your data *very* well in order to build the most robust system.

For this toy example, we'll just split blindly on length. 

>There's an opportunity to clear up some terminology here, for this course we will be stick to the following: 
>
>- "source documents" : The `.txt`, `.pdf`, `.html`, ..., files that make up the files and information we start with in its raw format
>- "document(s)" : single (or more) text object(s)
>- "corpus" : the combination of all of our documents

Let's take a peek visually at what we're doing here - and why it might be useful:

<img src="https://i.imgur.com/rtM6Ci6.png" />

As you can see (though it's not specifically true in this toy example) the idea of splitting documents is to break them into managable sized chunks that retain the most relevant local context.

In [10]:
text_splitter = CharacterTextSplitter()
split_documents = text_splitter.split_texts(documents)
len(split_documents)

189

Let's take a look at some of the documents we've managed to split.

In [11]:
for i in split_documents[0:1]:
    print(i)
    print("="*100)

ACT I
SCENE I. King Lear's palace.
Enter KENT, GLOUCESTER, and EDMUND
KENT
I thought the king had more affected the Duke of
Albany than Cornwall.
GLOUCESTER
It did always seem so to us: but now, in the
division of the kingdom, it appears not which of
the dukes he values most; for equalities are so
weighed, that curiosity in neither can make choice
of either's moiety.
KENT
Is not this your son, my lord?
GLOUCESTER
His breeding, sir, hath been at my charge: I have
so often blushed to acknowledge him, that now I am
brazed to it.
KENT
I cannot conceive you.
GLOUCESTER
Sir, this young fellow's mother could: whereupon
she grew round-wombed, and had, indeed, sir, a son
for her cradle ere she had a husband for her bed.
Do you smell a fault?
KENT
I cannot wish the fault undone, the issue of it
being so proper.
GLOUCESTER
But I have, sir, a son by order of law, some year
elder than this, who yet is no dearer in my account:
though this knave came something saucily into the
world before he was se


### Embeddings and Vectors

Next, we have to convert our corpus into a "machine readable" format. 

Loosely, this means turning the text into numbers. 

There are plenty of resources that talk about this process in great detail - I'll leave this [blog](https://txt.cohere.com/sentence-word-embeddings/) from Cohere:AI as a resource if you want to deep dive a bit. 

Today, we're going to talk about the actual process of creating, and then storing, these embeddings, and how we can leverage that to intelligently add context to our queries.

While this is all baked into 1 call - let's look at some of the code that powers this process:

Let's look at our `VectorDatabase().__init__()`:

```python
def __init__(self, embedding_model: EmbeddingModel = None):
        self.vectors = defaultdict(np.array)
        self.embedding_model = embedding_model or EmbeddingModel()
```

As you can see - our vectors are merely stored as a dictionary of `np.array` objects.

Secondly, our `VectorDatabase()` has a default `EmbeddingModel()` which is a wrapper for OpenAI's `text-embedding-ada-002` model. 

> **Quick Info About `text-embedding-ada-002`**:
> - It has a context window of **8192** tokens
> - It returns vectors with dimension **1536**

We can call the `async_get_embeddings` method of our `EmbeddingModel()` on a list of `str` and receive a list of `float` back!

```python
async def async_get_embeddings(self, list_of_text: List[str]) -> List[List[float]]:
        return await aget_embeddings(
            list_of_text=list_of_text, engine=self.embeddings_model_name
        )
```

We cast those to `np.array` when we build our `VectorDatabase()`:

```python
async def abuild_from_list(self, list_of_text: List[str]) -> "VectorDatabase":
        embeddings = await self.embedding_model.async_get_embeddings(list_of_text)
        for text, embedding in zip(list_of_text, embeddings):
            self.insert(text, np.array(embedding))
        return self
```

And that's all we need to do!

In [12]:
vector_db = VectorDatabase()
vector_db = asyncio.run(vector_db.abuild_from_list(split_documents))

In [15]:
vector_db.search_by_text("King learn",k=10)

[("great patron thought on in my prayers,--\nKING LEAR\nThe bow is bent and drawn, make from the shaft.\nKENT\nLet it fall rather, though the fork invade\nThe region of my heart: be Kent unmannerly,\nWhen Lear is mad. What wilt thou do, old man?\nThink'st thou that duty shall have dread to speak,\nWhen power to flattery bows? To plainness honour's bound,\nWhen majesty stoops to folly. Reverse thy doom;\nAnd, in thy best consideration, cheque\nThis hideous rashness: answer my life my judgment,\nThy youngest daughter does not love thee least;\nNor are those empty-hearted whose low sound\nReverbs no hollowness.\nKING LEAR\nKent, on thy life, no more.\nKENT\nMy life I never held but as a pawn\nTo wage against thy enemies; nor fear to lose it,\nThy safety being the motive.\nKING LEAR\nOut of my sight!\nKENT\nSee better, Lear; and let me still remain\nThe true blank of thine eye.\nKING LEAR\nNow, by Apollo,--\nKENT\nNow, by Apollo, king,\nThou swear'st thy gods in vain.\nKING LEAR\nO, vassal

So, to review what we've done so far in natural language:

1. We load source documents
2. We split those source documents into smaller chunks (documents)
3. We send each of those documents to the `text-embedding-ada-002` OpenAI API endpoint
4. We store each of the text representations with the vector representations as keys/values in a dictionary

### Semantic Similarity

The next step is to be able to query our `VectorDatabase()` with a `str` and have it return to us vectors and text that is most relevant from our corpus. 

We're going to use the following process to achieve this in our toy example:

1. We need to embed our query with the same `EmbeddingModel()` as we used to construct our `VectorDatabase()`
2. We loop through every vector in our `VectorDatabase()` and use a distance measure to compare how related they are
3. We return a list of the top `k` closest vectors, with their text representations

There's some very heavy optimization that can be done at each of these steps - but let's just focus on the basic pattern in this notebook.

> We are using [cosine similarity](https://www.engati.com/glossary/cosine-similarity) as a distance measure in this example - but there are many many distance measures you could use - like [these](https://flavien-vidal.medium.com/similarity-distances-for-natural-language-processing-16f63cd5ba55)

> We are using a rather inefficient way of calculating relative distance between the query vector and all other vectors - there are more advanced approaches that are much more efficient, like [ANN](https://towardsdatascience.com/comprehensive-guide-to-approximate-nearest-neighbors-algorithms-8b94f057d6b6)

In [16]:
vector_db.search_by_text("Your servant Kent. Where is your servant Caius?", k=3)

[("ng] O my good master!\nKING LEAR\nPrithee, away.\nEDGAR\n'Tis noble Kent, your friend.\nKING LEAR\nA plague upon you, murderers, traitors all!\nI might have saved her; now she's gone for ever!\nCordelia, Cordelia! stay a little. Ha!\nWhat is't thou say'st? Her voice was ever soft,\nGentle, and low, an excellent thing in woman.\nI kill'd the slave that was a-hanging thee.\nCaptain\n'Tis true, my lords, he did.\nKING LEAR\nDid I not, fellow?\nI have seen the day, with my good biting falchion\nI would have made them skip: I am old now,\nAnd these same crosses spoil me. Who are you?\nMine eyes are not o' the best: I'll tell you straight.\nKENT\nIf fortune brag of two she loved and hated,\nOne of them we behold.\nKING LEAR\nThis is a dull sight. Are you not Kent?\nKENT\nThe same,\nYour servant Kent: Where is your servant Caius?\nKING LEAR\nHe's a good fellow, I can tell you that;\nHe'll strike, and quickly too: he's dead and rotten.\nKENT\nNo, my good lord; I am the very man,--\nKING LEA

# Prompts

In the following section, we'll be looking at the role of prompts - and how they help us to guide our application in the right direction.

In this notebook, we're going to rely on the idea of "zero-shot in-context learning".

This is a lot of words to say: "We will ask it to perform our desired task in the prompt, and provide no examples."

### XYZRolePrompt

Before we do that, let's stop and think a bit about how OpenAI's chat models work. 

We know they have roles - as is indicated in the following API [documentation](https://platform.openai.com/docs/api-reference/chat/create#chat/create-messages)

There are three roles, and they function as follows (taken directly from [OpenAI](https://platform.openai.com/docs/guides/gpt/chat-completions-api)): 

- `{"role" : "system"}` : The system message helps set the behavior of the assistant. For example, you can modify the personality of the assistant or provide specific instructions about how it should behave throughout the conversation. However note that the system message is optional and the model’s behavior without a system message is likely to be similar to using a generic message such as "You are a helpful assistant."
- `{"role" : "user"}` : The user messages provide requests or comments for the assistant to respond to.
- `{"role" : "assistant"}` : Assistant messages store previous assistant responses, but can also be written by you to give examples of desired behavior.

The main idea is this: 

1. You start with a system message that outlines how the LLM should respond, what kind of behaviours you can expect from it, and more
2. Then, you can provide a few examples in the form of "assistant"/"user" pairs
3. Then, you prompt the model with the true "user" message.

In this example, we'll be forgoing the 2nd step for simplicities sake.

In [17]:
from aimakerspace.openai_utils.prompts import (
    UserRolePrompt,
    SystemRolePrompt,
    AssistantRolePrompt,
)

from aimakerspace.openai_utils.chatmodel import ChatOpenAI

chat_openai = ChatOpenAI()
user_prompt_template = "{content}"
user_role_prompt = UserRolePrompt(user_prompt_template)
system_prompt_template = (
    "You are an expert in {expertise}, you always answer in a kind way."
)
system_role_prompt = SystemRolePrompt(system_prompt_template)

messages = [
    user_role_prompt.create_message(
        content="What is the best way to write a loop?"
    ),
    system_role_prompt.create_message(expertise="Python"),
]

response = chat_openai.run(messages)

In [18]:
print(response)

The best way to write a loop depends on the specific situation and the programming language you are using. However, I can provide some general tips for writing loops efficiently and effectively:

1. Choose the appropriate type of loop: Depending on the nature of your task, you can use different types of loops such as "for" loop, "while" loop, or "do-while" loop. Make sure to select the loop type that best fits your problem.

2. Define your loop condition: Determine the condition under which the loop should continue executing. This could be based on a specific value, a certain range, or a boolean expression.

3. Initialize loop variables correctly: If you are using variables within your loop, ensure that they are initialized properly before the loop begins, especially if you're using a "while" loop.

4. Update loop variables: Inside the loop, update the loop variables according to the desired logic. This could involve incrementing or decrementing a counter, modifying a specific variable

### Retrieval Augmented Question Answering Prompt

Now we can create a RAQA prompt - which will help our system behave in a way that makes sense!

There is much you could do here, many tweaks and improvements to be made!

In [19]:
RAQA_PROMPT_TEMPLATE = """
Use the provided context to answer the user's query. 

You may not answer the user's query unless there is specific context in the following text.

If you do not know the answer, or cannot answer, please respond with "I don't know".

Context:
{context}
"""

raqa_prompt = SystemRolePrompt(RAQA_PROMPT_TEMPLATE)

USER_PROMPT_TEMPLATE = """
User Query:
{user_query}
"""

user_prompt = UserRolePrompt(USER_PROMPT_TEMPLATE)

class RetrievalAugmentedQAPipeline:
    def __init__(self, llm: ChatOpenAI(), vector_db_retriever: VectorDatabase) -> None:
        self.llm = llm
        self.vector_db_retriever = vector_db_retriever

    def run_pipeline(self, user_query: str) -> str:
        context_list = self.vector_db_retriever.search_by_text(user_query, k=4)
        
        context_prompt = ""
        for context in context_list:
            context_prompt += context[0] + "\n"

        formatted_system_prompt = raqa_prompt.create_message(context=context_prompt)

        formatted_user_prompt = user_prompt.create_message(user_query=user_query)
        
        return self.llm.run([formatted_system_prompt, formatted_user_prompt])

In [20]:
retrieval_augmented_qa_pipeline = RetrievalAugmentedQAPipeline(
    vector_db_retriever=vector_db,
    llm=chat_openai
)

In [21]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is King Lear?")

'The context provided indicates that King Lear is a character in the play. However, it does not provide any specific information about who King Lear is or his background.'

In [22]:
retrieval_augmented_qa_pipeline.run_pipeline("Who is Batman?")

"I don't know."

In [23]:
retrieval_augmented_qa_pipeline.run_pipeline("What happens to Cordelia?")

"Cordelia dies. She is brought into the scene dead in King Lear's arms. The text does not provide specific details about how she died."

# Conclusion

In this notebook, we've gone through the steps required to create your own simple RAQA application!

Please feel free to extend this as much as you'd like. 

# Bonus Challenges

Challenge 1: 
- Implement a new distance measure
- Implement a more efficient vector search

Challenge 2: 
- Create an external VectorStore that can be run/hosted elsewhere
- Build an adapter for that VectorStore here